In [ ]:
!pip install openpyxl tqdm scikit-learn datasets transformers evaluate torch rouge_score --quiet

In [ ]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-

"""
1단계: ‘지원대상’과 ‘지원내용’ 컬럼별로
  1) 불필요한 태그·기호 제거(clean_text)
  2) 중복되는 단어(연속·전체 반복) 축소
  3) 초벌 요약(EbanLee/kobart-summary-v3)
     (빈 요약 → 원문(cleaned) 대체)
  4) 이미 요약된 행 건너뛰기
- INTERIM_XLSX에 결과 저장
"""

from google.colab import drive  # Colab 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

import os
import sys
import re
import pandas as pd
from tqdm import tqdm
from transformers import AutoTokenizer, BartForConditionalGeneration
import torch

# ─── 경로 설정 ─────────────────────────────────────────────────────────
BASE_PATH    = '/content/drive/MyDrive'  # 드라이브 내 작업 경로
INPUT_XLSX   = os.path.join(BASE_PATH, "여성맞춤정책_분류+상세_v6.xlsx")
INTERIM_XLSX = os.path.join(BASE_PATH, "여성맞춤정책_요약_2차_결과.xlsx")

# ─── 디바이스 설정 ─────────────────────────────────────────────────────
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")  # GPU 사용 우선

# ─── 전처리용 정규식 정의 ─────────────────────────────────────────────
BULLET_RE      = re.compile(r"^[\s\t]*[ㅇ○●◆■▪▶►△▲→←‣·•※…☆★▶▷▸▹▻▵▴➤➜❖]+[\s\t]*")  # 글머리 기호 제거
HEADER_RE      = re.compile(r'^\s*(지원대상|지원내용|지원방식|지원금액)\s*[:은는]?\s*')  # 컬럼 라벨 제거
MULTI_SPACE_RE = re.compile(r'\s{2,}')  # 연속 공백 -> 단일 공백
TAG_RE         = re.compile(r"<.*?>")  # HTML 태그 제거
DUP_WORD_RE    = re.compile(r'\b(\w+)(?:\s+\1)+\b', flags=re.IGNORECASE)  # 반복 단어 축소
PAREN_NUM_RE   = re.compile(r'^\(\d+\)\s*')  # (숫자) 제거
CIRCLE_NUM_RE  = re.compile(r'^[①-⑳]\s*')  # 원형 번호(①~⑳) 제거

# ─── 텍스트 정제 함수 ─────────────────────────────────────────────────
def clean_text(txt: str) -> str:
    """
    1) TAG_RE로 HTML 태그 제거
    2) HEADER_RE로 컬럼 라벨(지원대상:/지원내용:) 제거
    3) 줄바꿈을 공백으로 변환 후 MULTI_SPACE_RE로 중복 공백 제거
    4) DUP_WORD_RE로 연속 반복 단어 축소
    """
    t = TAG_RE.sub("", str(txt))
    t = HEADER_RE.sub("", t)
    t = MULTI_SPACE_RE.sub(" ", t.replace("\n", " ")).strip()
    t = DUP_WORD_RE.sub(r'\1', t)
    return t

# ─── 요약 후처리 함수 ─────────────────────────────────────────────────
def postprocess_summary(txt: str, fallback: str) -> str:
    """
    1) 모델 토큰 "</s>" 제거
    2) 빈 문자열인 경우 fallback(원문) 사용 → 요약이 빈 행인 경우 해결
    3) 마침표/합니다 체 보장, 없으면 "을(를) 지원합니다." 추가
    """
    s = txt.replace("</s>", "").strip()
    if not s:
        return fallback  # 빈 요약 → 원문 대체
    if not s.endswith((".", "니다", "습니다", "요", "음")):
        s += "을(를) 지원합니다."
    return s

# ─── 모델 로드 ───────────────────────────────────────────────────────
MODEL_NAME = "EbanLee/kobart-summary-v3"
tokenizer  = AutoTokenizer.from_pretrained(MODEL_NAME, use_fast=False)
model      = BartForConditionalGeneration.from_pretrained(MODEL_NAME).to(DEVICE)

# ─── 데이터 로드 ─────────────────────────────────────────────────────
try:
    df = pd.read_excel(INPUT_XLSX, engine="openpyxl")
except FileNotFoundError:
    sys.exit(f"❌ 입력 파일을 찾을 수 없습니다: {INPUT_XLSX}")

# ─── 컬럼 초기화 ─────────────────────────────────────────────────────
for col in ["지원대상_원문", "지원대상_초벌요약", "지원내용_원문", "지원내용_초벌요약"]:
    if col in df.columns:
        df[col] = df[col].fillna("")  # Series만 fillna 적용 → AttributeError 방지
    else:
        df[col] = ""  # 없는 컬럼은 새로 생성

# ─── 요약 루프 ─────────────────────────────────────────────────────────
for idx, row in tqdm(df.iterrows(), total=len(df), desc="1단계 요약"):
    # 이미 요약된 경우 건너뛰어 중복 작업 방지
    if row["지원대상_초벌요약"].strip() and row["지원내용_초벌요약"].strip():
        continue

    # ── 지원대상 원문 처리 ─────────────────────────────────────────
    raw = str(row.get("지원대상", ""))
    # 레이블 통일: '지원대상:' 및 '대상자:' → '해당자:'
    raw = re.sub(r'\b지원대상:', '해당자:', raw)
    raw = re.sub(r'\b대상자?:',  '해당자:', raw)

    cnt = 0
    lines_target = []
    for line in raw.splitlines():
        t = line.strip()
        if not t:
            continue  # 빈 라인 건너뜀
        if t.startswith('-'):
            # 하이픈 시작 → 서브 항목 유지
            sub = t.lstrip('- ').strip()
            cleaned = clean_text(sub)
            if cleaned:
                lines_target.append(f"- {cleaned}")
        else:
            # (숫자), ①~ 제거 및 불릿 제거 → clean_text → 빈 방지
            t0 = PAREN_NUM_RE.sub("", t)
            t0 = CIRCLE_NUM_RE.sub("", t0)
            t0 = BULLET_RE.sub("", t0)
            cleaned = clean_text(t0)
            if not cleaned:
                continue  # 내용 없는 번호 줄 제거
            cnt += 1
            lines_target.append(f"({cnt}){cleaned}")  # 중복 번호 방지

    df.at[idx, "지원대상_원문"] = "\n".join(lines_target)

    # ── 지원내용 원문 처리 ─────────────────────────────────────────
    raw = str(row.get("지원내용", ""))
    raw = re.sub(r'\b지원내용:', '세부사항:', raw)  # 레이블 통일
    raw = re.sub(r'\b내용:',     '세부사항:', raw)

    cnt = 0
    lines_content = []
    for line in raw.splitlines():
        t = line.strip()
        if not t:
            continue
        if t.startswith('-'):
            sub = t.lstrip('- ').strip()
            cleaned = clean_text(sub)
            if cleaned:
                lines_content.append(f"- {cleaned}")
        else:
            t0 = PAREN_NUM_RE.sub("", t)
            t0 = CIRCLE_NUM_RE.sub("", t0)
            t0 = BULLET_RE.sub("", t0)
            cleaned = clean_text(t0)
            if not cleaned:
                continue
            cnt += 1
            lines_content.append(f"({cnt}){cleaned}")

    df.at[idx, "지원내용_원문"] = "\n".join(lines_content)

    # ── 지원대상 초벌 요약 ─────────────────────────────────────────
    summary_target = " ".join(l[3:] for l in lines_target if l.startswith('('))
    if summary_target:
        inp = tokenizer(summary_target, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
        out = model.generate(
            **inp,
            min_length=10,
            max_length=128,
            num_beams=5,
            repetition_penalty=2.0,
            no_repeat_ngram_size=2,  # 2-gram 중복 차단으로 반복문구 제거
            length_penalty=1.2,      # 지나치게 짧은 요약 방지
            early_stopping=True
        )
        dec = tokenizer.decode(out[0], skip_special_tokens=True)
        # 요약 내 레이블 치환
        dec = re.sub(r'\b지원대상:', '해당자:', dec)
        dec = re.sub(r'\b대상자?:',  '해당자:', dec)
        df.at[idx, "지원대상_초벌요약"] = postprocess_summary(dec, summary_target)
    else:
        df.at[idx, "지원대상_초벌요약"] = summary_target  # 빈행이면 원문 대체

    # ── 지원내용 초벌 요약 ─────────────────────────────────────────
    summary_content = " ".join(l[3:] for l in lines_content if l.startswith('('))
    if summary_content:
        inp = tokenizer(summary_content, return_tensors="pt", truncation=True, max_length=1024).to(DEVICE)
        out = model.generate(
            **inp,
            min_length=10,
            max_length=128,
            num_beams=5,
            repetition_penalty=2.0,
            no_repeat_ngram_size=2,
            length_penalty=1.2,
            early_stopping=True
        )
        dec = tokenizer.decode(out[0], skip_special_tokens=True)
        dec = re.sub(r'\b지원내용:', '세부사항:', dec)
        dec = re.sub(r'\b내용:',     '세부사항:', dec)
        df.at[idx, "지원내용_초벌요약"] = postprocess_summary(dec, summary_content)
    else:
        df.at[idx, "지원내용_초벌요약"] = summary_content  # 빈행이면 원문 대체

# ── 중간 결과 저장 ───────────────────────────────────────────────────
df.to_excel(INTERIM_XLSX, index=False, engine="openpyxl")
print(f"✅ 1단계 완료: {INTERIM_XLSX}")

Mounted at /content/drive


You passed `num_labels=3` which is incompatible to the `id2label` map of length `2`.
1단계 요약: 100%|██████████| 1292/1292 [15:55<00:00,  1.35it/s]


✅ 1단계 완료: /content/drive/MyDrive/여성맞춤정책_요약_2차_결과.xlsx


In [2]:
!pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 172.3/172.3 kB 3.1 MB/s eta 0:00:00


In [3]:
import re
import os
import pandas as pd

from google.colab import drive  # Colab 드라이브 마운트
drive.mount('/content/drive', force_remount=True)

BASE_PATH    = '/content/drive/MyDrive'  # 드라이브 내 작업 경로
IN_PATH = os.path.join(BASE_PATH,"여성맞춤정책_요약_2차_결과.xlsx")

def fix_bullet_lines(text: str) -> str:
    lines = text.splitlines()
    fixed = []
    for line in lines:
        m = re.match(r'^(\s*)-(\s*)(.*)$', line)
        if m:
            content = m.group(3)
            fixed.append(f" - {content.strip()}")
        else:
            fixed.append(line)
    return "\n".join(fixed)

def fix_parentheses_spacing(text: str) -> str:
    s = text

    # --- [A] 숫자만 들어있는 괄호는 플레이스홀더로 대체 (내부는 붙여쓰기 유지) ---
    placeholder_fmt = "<<<P_NUM:{}>>>"
    nums = []

    def _num_hold(m):
        num = m.group(1)
        idx = len(nums)
        nums.append(num)
        return placeholder_fmt.format(idx)

    s = re.sub(r'\(\s*(\d+)\s*\)', _num_hold, s)

    # --- [B] 일반 괄호 내부 공백 정리: "(내용)" -> "( 내용 )" ---
    s = re.sub(r'\(\s*([^\)]*?)\s*\)', r'( \1 )', s)

    # --- [C] 괄호 외부 공백 정리(일반 괄호에만 적용됨) ---
    s = re.sub(r'(?P<prev>\S)\(', r'\g<prev> (', s)  # 앞쪽
    s = re.sub(r'\)(?P<next>\S)', r') \g<next>', s)  # 뒤쪽

    # --- [D] 숫자 괄호 복원 ---
    def _num_restore(m):
        idx = int(m.group(1))
        return f'({nums[idx]})'
    s = re.sub(r'<<<P_NUM:(\d+)>>>', _num_restore, s)

    # --- [E] 숫자 괄호의 외부 공백 보정 (복원 후 적용) ---
    # 앞쪽: 붙은문자(숫자/한글/영문/%) + "(숫자)" -> "문자 (숫자)"
    s = re.sub(r'(?P<prev>\S)\((\d+)\)', r'\g<prev> (\2)', s)
    # 뒤쪽: "(숫자)문자" -> "(숫자) 문자"
    s = re.sub(r'\((\d+)\)(?P<next>\S)', r'(\1) \g<next>', s)

    # --- [F] 라인별 다중 공백 축소(선행 들여쓰기는 유지) ---
    lines = []
    for line in s.splitlines():
        line = line.replace('\t', ' ')
        m = re.match(r'^(\s*)(.*)$', line)
        lead, body = m.group(1), m.group(2)
        body = re.sub(r' {2,}', ' ', body)
        lines.append(lead + body)
    return "\n".join(lines)

def process_cell(val):
    if pd.isna(val):
        return val
    t = str(val)
    t = fix_bullet_lines(t)
    t = fix_parentheses_spacing(t)
    return t

# 모든 시트/모든 열 처리
xls = pd.read_excel(IN_PATH, sheet_name=None)
out_sheets = {}
for sheet_name, df in xls.items():
    df2 = df.copy()
    for col in df2.columns:
        if df2[col].dtype == object:
            df2[col] = df2[col].map(process_cell)
    out_sheets[sheet_name] = df2

# Google Drive 경로로 저장
OUT_PATH = os.path.join(BASE_PATH, "여성맞춤정책_요약_2차_결과_정리.xlsx")

with pd.ExcelWriter(OUT_PATH, engine="xlsxwriter") as writer:
    for sheet_name, df in out_sheets.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)

df.head(3)

Mounted at /content/drive


,대상유형,지역,제목,detail_url,신청기간,신청방법,접수기관,지원형태,지원대상,지원내용,문의처,기타,카테고리_분류,지원형태_분류,지원대상_원문,지원대상_초벌요약,지원내용_원문,지원내용_초벌요약
0,한부모,강원특별자치도,강릉의료원 보건의료 복지 통합지원 서비스,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/O000...,상시신청,○ 기타\n - 전자공문,NaN,"현금 ( 감면 ) , 서비스 ( 의료 )","지원대상\n○ 국민기초생활수급자 ( 생계, 의료, 주거, 교육 급여 )\n\n○ 차...",지원내용\n○ 의료비 지원\n - 질병치료와 관련한 강릉의료원 의료서비스 이용시 급...,보건의료복지통합지원팀 ( ☎033-610-1492 ),NaN,취약계층,"직접금전지원,감면할인지원,의료지원,서비스","(1) 국민기초생활수급자 ( 생계, 의료, 주거, 교육 급여 )\n(2) 차상위계층...","국민기초생활수급자 ( 생계, 의료, 주거, 교육 급여 ) 차상위계층 가구건강보험료 ...",(1) 의료비 지원\n - 질병치료와 관련한 강릉의료원 의료서비스 이용시 급여 본인...,의료비 및 만성질환 관리 및 모니터링
1,한부모,강원특별자치도,강원형 수선유지 주거급여 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/6420...,접수기관 별 상이,○ 방문 신청\n - 주민센터 : 주소지 읍면동 주민센터에 방문 신청\n - 시군 ...,"주민센터,시·군·구청",현물,"지원대상\n○ 기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소년·소녀가...","지원내용\n○ 지원대상\n - 기준 중위소득 50%이하 차상위계층 중 고령자, 장애...",건축과 ( ☎033-249-3464 ),NaN,"한부모,고령자,취약계층","주거지원,물품지원","(1) 기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소년·소녀가정, 다...","기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소년·소녀가정, 다자녀 가...","(1) 해당사항\n - 기준 중위소득 50%이하 차상위계층 중 고령자, 장애인, 소...",가구당 최대 5백만원 지원 등
2,한부모,강원특별자치도,방학 중 아동급식 지원,https://www.gov.kr/portal/rcvfvrSvc/dtlEx/6530...,상시신청,거주지 관할 읍/면/동 행정복지센터에 방문 신청,주민센터,현물,지원대상\n(1) 아래의 어느 하나에 해당하는 아동으로서 결식우려가 있는 아동\n ...,"지원내용\n○ 지원연령: 18세 미만의 취학 아동 ( 초등학교, 중학교, 고등학교 ...",복지정책과 ( ☎033-249-2267 ),NaN,"한부모,임신/출산/육아,초/중/고등학생,취약계층","교육지원,물품지원",(1) 아래의 어느 하나에 해당하는 아동으로서 결식우려가 있는 아동\n - 국민기초...,아동복지프로그램을 통해 식사를 제공받는 경우 해당 식사분에 대하여 급식카드 등 다른...,"(1) 연령: 18세 미만의 취학 아동 ( 초등학교, 중학교, 고등학교 재학 ) 및...","18세 미만의 취학 아동 및 학교밖 청소년 지원유형은 아동급식카드, 지역아동센터지원..."
